In [4]:
# Installing requirements
%pip install datasets
%pip install fsspec
%pip install transformers
!python -m pip install --upgrade transformers
%pip install torch
%pip install tensorflow
%pip install flax
%pip install sentencepiece
%pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import necessary libraries
from datasets import load_dataset, DatasetDict
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, pipeline
import ipywidgets as widgets
from IPython.display import display

In [6]:
# Explore and preprocess the dataset
dataset = load_dataset("Kaludi/Customer-Support-Responses")
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'response'],
        num_rows: 74
    })
})

In [7]:
# Choose a pre-trained model tokenizer
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_data(example):
    inputs = tokenizer(example['query'], padding='max_length', truncation=True, max_length=512)
    outputs = tokenizer(example['response'], padding='max_length', truncation=True, max_length=512)
    inputs['labels'] = outputs['input_ids']
    return inputs

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Apply tokenization
tokenized_dataset = dataset.map(tokenize_data, batched=True)

# Split the dataset
num_train = int(0.7 * len(tokenized_dataset['train']))
num_valid = int(0.3 * len(tokenized_dataset['train']))
num_test = len(tokenized_dataset['train']) - num_train - num_valid

train_dataset = tokenized_dataset['train'].select(range(num_train))
valid_dataset = tokenized_dataset['train'].select(range(num_train, num_train + num_valid))
test_dataset = tokenized_dataset['train'].select(range(num_train + num_valid, len(tokenized_dataset['train'])))

splits = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

# Display the splits
print(splits)

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['query', 'response', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 51
    })
    validation: Dataset({
        features: ['query', 'response', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
    test: Dataset({
        features: ['query', 'response', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})


In [6]:
# Load pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=splits['train'],
    eval_dataset=splits['validation'],
)

# Train the model
trainer.train()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,12.553866
2,No log,11.376226
3,No log,10.932395


TrainOutput(global_step=12, training_loss=12.145262400309244, metrics={'train_runtime': 16.9728, 'train_samples_per_second': 9.014, 'train_steps_per_second': 0.707, 'total_flos': 20707295625216.0, 'train_loss': 12.145262400309244, 'epoch': 3.0})

In [7]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('./fine-tuned-t5')
tokenizer.save_pretrained('./fine-tuned-t5')

# Load the fine-tuned model and tokenizer
fine_tuned_model = T5ForConditionalGeneration.from_pretrained('./fine-tuned-t5')
fine_tuned_tokenizer = T5Tokenizer.from_pretrained('./fine-tuned-t5')

# Define the pipeline for text generation
text_generator = pipeline('text2text-generation', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Define a function to generate responses
def generate_response(query):
    response = text_generator(query, max_length=50, num_return_sequences=1)
    return response[0]['generated_text']

# Create interactive widgets
query_widget = widgets.Text(
    value='',
    placeholder='Enter your query here',
    description='Query:',
    disabled=False
)

response_widget = widgets.Textarea(
    value='',
    placeholder='Response will be displayed here',
    description='Response:',
    disabled=True,
    layout=widgets.Layout(width='100%', height='200px')
)

button = widgets.Button(
    description='Generate Response',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to generate response',
    icon='check'  # (FontAwesome names without the `fa-` prefix)
)

# Define the button click event
def on_button_click(b):
    query = query_widget.value
    response = generate_response(query)
    response_widget.value = response

button.on_click(on_button_click)

# Display the widgets
display(query_widget, button, response_widget)

Text(value='', description='Query:', placeholder='Enter your query here')

Button(description='Generate Response', icon='check', style=ButtonStyle(), tooltip='Click to generate response…

Textarea(value='', description='Response:', disabled=True, layout=Layout(height='200px', width='100%'), placeh…